In [1]:
import sys
import os

# Get the current working directory of the notebook
notebook_dir = os.getcwd()

# Define the root directory
root_dir = os.path.abspath(os.path.join(notebook_dir, '..', '..', '..'))

# Add the root directory to sys.path
sys.path.insert(0, root_dir)

# set environment variable 'GRETEL_PROD_API_KEY'
os.environ['GRETEL_PROD_API_KEY'] = 'your_api_key'

In [16]:
# I'm having trouble finding my virtual environment from within the notebook, so installing all the requirements here
# !pip install -r /mnt/foundation-shared/nina_xu_gretel_ai/navigator-helpers/requirements.txt

# Restart the kernel if needed

In [ ]:
# Reload packages
# Run this cell if you've made changes to the evaluation.py file 
# Alternatively, you can restart the kernel to pick up changes
from importlib import reload
import evaluation
reload(evaluation)

In [30]:
import pandas as pd

from navigator_helpers.llms.llm_suite import GretelLLMSuite

from evaluation import BaseEvaluationTaskSuite, NL2SQLEvaluationTaskSuite

from datasets import load_dataset

In [24]:
## TODO: add dict of datasets to test
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")
dataset_10 = dataset.select(range(10))
dataset_10_pd = dataset_10.to_pandas()

dataset_1_pd = dataset.select(range(1)).to_pandas()

In [ ]:
sql_linter_results = pd.read_csv("/mnt/foundation-shared/nina_xu_gretel_ai/datasets/sqlqueries_1200_validated_092524.csv")

In [ ]:
llm_suite = GretelLLMSuite()

In [ ]:

results_1 = BaseEvaluationTaskSuite(llm_suite, dataset_10_pd).row_uniqueness()
results_2 = BaseEvaluationTaskSuite(llm_suite, dataset_10_pd).feature_cardinality()
results_3 = BaseEvaluationTaskSuite(llm_suite, dataset_10_pd).feature_distribution()
results_4 = BaseEvaluationTaskSuite(llm_suite, dataset_10_pd).num_words_per_record()

In [ ]:
print(results_1)
print(results_2)
print(results_3)
print(results_4)

In [ ]:
task_5 = NL2SQLEvaluationTaskSuite(
    llm_suite=llm_suite, dataset=dataset_10_pd, code_lang="sql"
    )
results_5 = task_5.llm_as_a_critic_evaluation(
    instruction_col_name="sql_prompt", code_col_name="sql", context_col_name="sql_context"
)
table5 = task_5.output_dataset

# task_6 = BaseEvaluationTaskSuite(llm_suite, dataset_10_pd)
# results_6 = task_6.llm_as_a_critic_evaluation(
#     instruction_col_name="sql_prompt", code_col_name="sql"
# )

In [26]:

print(results_5)
# print(results_6)

# review specific records
# print(dataset_10_pd.loc[results_1['non_semantically_unique_ids']])

{'llm_as_a_critic_score': 3.5800000000000005}


In [28]:
table5

,id,domain,domain_description,sql_complexity,sql_complexity_description,sql_task_type,sql_task_type_description,sql_prompt,sql_context,sql,sql_explanation,scores,overall_score
0,5097,forestry,Comprehensive data on sustainable forest manag...,single join,"only one join (specify inner, outer, cross)",analytics and reporting,"generating reports, dashboards, and analytical...",What is the total volume of timber sold by eac...,"CREATE TABLE salesperson (salesperson_id INT, ...","SELECT salesperson_id, name, SUM(volume) as to...","Joins timber_sales and salesperson tables, gro...","{'relevance_score': 4, 'relevance_reason': 'Th...",3.6
1,5098,defense industry,"Defense contract data, military equipment main...",aggregation,"aggregation functions (COUNT, SUM, AVG, MIN, M...",analytics and reporting,"generating reports, dashboards, and analytical...",List all the unique equipment types and their ...,CREATE TABLE equipment_maintenance (equipment_...,"SELECT equipment_type, SUM(maintenance_frequen...",This query groups the equipment_maintenance ta...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.6
2,5099,marine biology,"Comprehensive data on marine species, oceanogr...",basic SQL,basic SQL with a simple select statement,analytics and reporting,"generating reports, dashboards, and analytical...",How many marine species are found in the South...,"CREATE TABLE marine_species (name VARCHAR(50),...",SELECT COUNT(*) FROM marine_species WHERE loca...,This query counts the number of marine species...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.6
3,5100,financial services,Detailed financial data including investment s...,aggregation,"aggregation functions (COUNT, SUM, AVG, MIN, M...",analytics and reporting,"generating reports, dashboards, and analytical...",What is the total trade value and average pric...,"CREATE TABLE trade_history (id INT, trader_id ...","SELECT trader_id, stock, SUM(price * quantity)...",This query calculates the total trade value an...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.6
4,5101,energy,Energy market data covering renewable energy s...,window functions,"window functions (e.g., ROW_NUMBER, LEAD, LAG,...",analytics and reporting,"generating reports, dashboards, and analytical...",Find the energy efficiency upgrades with the h...,"CREATE TABLE upgrades (id INT, cost FLOAT, typ...","SELECT type, cost FROM (SELECT type, cost, ROW...",The SQL query uses the ROW_NUMBER function to ...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.6
5,5102,defense operations,"Defense data on military innovation, peacekeep...",basic SQL,basic SQL with a simple select statement,analytics and reporting,"generating reports, dashboards, and analytical...",What is the total spending on humanitarian ass...,CREATE SCHEMA if not exists defense; CREATE TA...,SELECT SUM(spending) FROM defense.eu_humanitar...,This SQL query calculates the total spending o...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.6
6,5103,aquaculture,"Aquatic farming data, fish stock management, o...",single join,"only one join (specify inner, outer, cross)",analytics and reporting,"generating reports, dashboards, and analytical...",What is the average water temperature for each...,"CREATE TABLE SpeciesWaterTemp (SpeciesID int, ...","SELECT SpeciesName, AVG(WaterTemp) as AvgTemp ...",This query calculates the average water temper...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.2
7,5104,nonprofit operations,"Donation records, program outcomes, volunteer ...",basic SQL,basic SQL with a simple select statement,data manipulation,"inserting, updating, or deleting records",Delete a program's outcome data,"CREATE TABLE Program_Outcomes (id INT, program...",DELETE FROM Program_Outcomes WHERE program_id ...,This query removes the record with a program_i...,"{'relevance_score': 4, 'relevance_reason': 'Th...",3.8
8,5105,public transportation,"Extensive data on route planning, fare collect...",basic SQL,basic SQL with a simple select stat

In [63]:
# Coalesce the results of the five dialects into a single column
sql_linter_results['is_valid_sql'] = sql_linter_results['is_valid_sqlite'].fillna(
    sql_linter_results['is_valid_mysql'].fillna(
        sql_linter_results['is_valid_postgresql'].fillna(
            sql_linter_results['is_valid_sqlserver'].fillna(
                sql_linter_results['is_valid_googlesql']
            )
        )
    )
)
sql_linter_results.is_valid_sql.value_counts()


is_valid_sql
True     917
False    147
Name: count, dtype: int64

In [27]:
print(table5['scores'].loc[0])


{'relevance_score': 4, 'relevance_reason': 'The query directly addresses the prompt and uses the provided tables.', 'correctness_score': 4, 'correctness_reason': 'The query returns the total volume of timber sold by each salesperson, sorted by salesperson.', 'readability_score': 3, 'readability_reason': 'The query is well-formatted and uses aliases, but lacks comments.', 'scalability_score': 3, 'scalability_reason': 'The query should scale well, but could be optimized with an index on salesperson_id.', 'standards_score': 4, 'standards_reason': 'The query follows SQL standards and best practices.'}


In [ ]:
table6 = task_6.output_dataset
print(table6['scores'].loc[0])